# Normalizing Messy Financial Data

As a Financial Analyst, most of my work takes place in Excel and one of my monthly routines is to consolidate financial information from various, remote business units; I achieve this by providing each business unit with an Excel template that is to be filled out. Unfortunately, since most data is user-inputted, I often receive excel templates that have typos, miscategorizations, and other errors that make consolidation a nightmare. Unfortunately,  excel isn't powerful enough to handle most unstructured data. However, we can leverage Python to identify and correct user-inputted errors fairly easily.

The packages that we'll use in this tutorial are the following:
- [pandas](http://pandas.pydata.org/)
 - Helps import and manipulate csv data
- [pandasql](https://pypi.python.org/pypi/pandasql)
 - Performs queries on csv data
- [fuzzywuzzy](https://pypi.python.org/pypi/fuzzywuzzy)
 - Assists with analyzing text data
- [openpyxl](https://openpyxl.readthedocs.io/en/default/)
 - Enables writing to excel

 

## The Data
Before we begin, it might be worth digging into the details of our problem and our proposed solution - taking time to plan may pay dividends in the near future. 

First, let us consider the incoming data. As previously mentioned, users are submitting financial data; the financial data includes the following information:

- Fiscal Year (i.e "2016"
- Fiscal Month (i.e "01" = January)
- Expense Detail (e.g "Board of Director Fees"... or in some cases "Broad Fees" [typo implied])
- Expense value ($)

While most of the aforementioned information is fairly straight forward, I often see errors within the "Expense Detail" field. The "Expense Detail" is merely a description of the type of expense - often, the type of expense is subjective to the accountants' interpretation. As a result, I often see similar but *slightly* nonstandardized expense categories (thus causing a problem during consolidation). 

## Our Approach
In order to reconcile variations in "Expense Details", we'll have to declare standard, `accepted` values. The `accepted` values will be the following:

- 'Severance'
- 'Legal Fees'
- 'Import & Export Fees'
- 'I.T. Fees'
- 'Board Fees'
- 'Acquisition Fees'

Each user-inputted Expense will be compared against the 'accepted' values. If the user inputs are closely related, we will replace the original, user-inputted value with our `accepted` value. 

Finally, we'll append the the `corrected_expense` into the original excel file so that we can pass along to our colleagues for reporting purposes :) 


## Let's begin
With our problem and approach defined, let's begin programming a solution. First, we'll want to import the necessary Python libraries we've discussed above.

In [226]:
#import necessary libraries
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from pandasql import sqldf
from openpyxl import load_workbook

## Importing the Data
Pandas has a plethora of functions that enable us to read and manipulate csv data. Below, we're using panda's `.read_excel` function to parse excel data into a workable format (known as a dataframe). 

In [228]:
#use pandas to read the excel file
xl = pd.read_excel(open("/Users/home/anaconda/envs/Consolidated_Expenses/data/expenses.xlsx",'rb'))

#let's view our data
xl

,Fiscal Year,Fiscal Month,Expense,Value
0,2016,1,Board of Director Fees,1215.089553
1,2016,1,Legal Fees,595.950391
2,2016,2,Import/Export,987.635522
3,2016,2,Severance,819.535131
4,2016,3,IT,119.974680
5,2016,3,Acquisition Costs,643.928766
6,2016,3,Board Fees,885.393901
7,2016,3,Legal,801.496901
8,2016,4,Import Export,634.432539
9,2016,5,Severance,347.268470


## Understanding the Data
As you can see above, there are many different "Expense" details. Unfortunately, the original datafile seems messy and cumbersome. Fortunately, we can leverage the `pandasql` library to help query the `pandas` dataframe (`xl`). 

The query below is a very simple SQL query that highlights all of the unique "Expense" details (and disregards any duplications). Additionally, the query sorts the data alphabetically for easier viewing.

In [148]:
#It seems like there are multiple, slightly similar categories of 'Expenses'
#Let's use SQL to pull each unique 'Expense' category
q = """
    SELECT DISTINCT Expense 
    FROM xl
    ORDER BY Expense ASC

"""

expense_types = sqldf(q)
print(expense_types)

                   Expense
0             Acquisition 
1        Acquisition Costs
2               Board Fees
3   Board of Director Fees
4         Company Acquired
5            Director Fees
6                     I.T.
7               I.T. Costs
8                       IT
9            Import Export
10           Import/Export
11                     Law
12                   Legal
13              Legal Fees
14              Severance 


## Introducing Standardization
As one could see, there are many variations of similar "Expense" Details (e.g "Import Export" vs "Import/Export"). While these variations are small, they are a hassle when trying to work with Excel pivot tables. In order to address the aforementioned issue, we need to create a standardized list of acceptable terms. 

In [229]:
#We will use these standard categories to identify and correct mislabeled expenses
accepted = ['Severance', 'Legal Fees', 'Import & Export Fees', 'I.T. Fees', 'Board Fees', 'Acquisition Fees']

## Comparing Original Data to our Accepted Standard
Here is where things get interesting; we'll be using the fuzzywuzzy library to compare text strings. 

Essentially, the fuzzywuzzy library tokenizes and rearranges each word within a string in attempt to find a match. Of course, not every string is *exactly* the same, therefore fuzzywuzzy will assign a score from 0 to 100 (where 100 is an absolute, perfect match). We will take advantage of fuzzywuzzy's scoring system to determine whether or not a user-inputted Expense matches our `accepted` values. 

**NOTE 1**: I've chosen to use the `token_set_ratio` method of fuzzywuzzy because word order and string length may vary in our original text files. However, there are [other methods](http://chairnerd.seatgeek.com/fuzzywuzzy-fuzzy-string-matching-in-python/) associated with fuzzywuzzy that might be more applicable depending on you specific needs.

**NOTE 2**: Below I've used the `.apply` method to iterate through our current dataframe. If I were to build a more robust application, I would define a function that set a threshold of acceptable fuzzywuzzy values (e.g any value over `30` would be accepted as a match). However, given my personal time constraint, I've foregone this process. 

In [230]:
correction_formula = lambda x: process.extractOne(x,accepted,scorer=fuzz.token_set_ratio)

corrected_expenses = xl['Expense'].apply(correction_formula)

corrected_df = corrected_expenses.to_frame('corrected expenses')
corrected_df

,corrected expenses
0,"(Board Fees, 100)"
1,"(Legal Fees, 100)"
2,"(Import & Export Fees, 100)"
3,"(Severance, 100)"
4,"(I.T. Fees, 40)"
5,"(Acquisition Fees, 81)"
6,"(Board Fees, 100)"
7,"(Legal Fees, 100)"
8,"(Import & Export Fees, 100)"
9,"(Severance, 100)"


## Writing the Transformed Data back to Excel
As you can see above, we've iterated through our original dataframe and assigned the best-matching, `accepted` value to the `corrected_df` dataframe. In order to wrap up the process, we need to export our finding back into the original excel file so that we can pass on a clean version of our work. 

Below we leverage `openpyxl`, a library used to write to excel!

In [225]:
book = load_workbook('expenses.xlsx')
writer = pd.ExcelWriter('expenses.xlsx', engine='openpyxl') 
writer.book = book
writer.sheets = dict((ws.title, ws) for ws in book.worksheets)

corrected_df.to_excel(writer, "Sheet1", startcol = 4)

writer.save()


## In Conclusion...
While this tutorial isn't earth-shattering, I hope that my work has opened up the eyes of other Financial Analysts who suffer from similar, data management problems. Furthermore, I hope I've piqued your interest of python and illustrated the capabilities Python has over Excel. Finally, please excuse my messy code - I am sure it can be improved (and I continuously strive to improve my programming skills) - given more time I could develop a much more robust process. However, currently the previously detailed process is sufficient for this tutorial's purpose :)

